# FmlandWthrInfo

- 작물명  →  (지역코드(AREA_ID), 작물특성코드(PA_CROP_SPE_ID))  ⇒  FmlandWthrInfo

## 기상청_작물별 농업주산지 상세날씨 조회서비스

- 주의사항 : 

- Data Source : [기상청_작물별 농업주산지 상세날씨 조회서비스 - 일통계조회](https://www.data.go.kr/data/15059518/openapi.do)

- Collection Method : API_REST

- Data Format : JSON+XML

- 참고문서 : 작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05.xlsx
    - 지역코드와 작물특성코드 목록

### 데이터 설명

- 농작물(36종) 주산지의 현재 날씨 및 동네예보 정보와 과거날씨 통계(일별, 순별, 월별) 정보 제공

### 요청변수(Request Parameter)

|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|---|---|---|---|---|---|
|서비스키|ServiceKey|4|필수|-|공공데이터포털에서 받은 인증키|
|페이지 번호|pageNo|4|필수|1|페이지번호|
|한 페이지 결과 수|numOfRows|4|필수|10|한 페이지 결과 수|
|응답자료형식|dataType|4|옵션|XML|요청자료형식(XML/JSON)|
|시작 연월일|ST_YMD|`0|필수|20161201|일통계 시작 날짜(YYYYMMDD)|
|종료 연월일|ED_YMD|10|필수|20161201|일통계 종료 날짜(YYYYMMDD)|
|지역 아이디|AREA_ID|10|필수|4122000000|지역 아이디(활용가이드 하단첨부 참고)|
|작물별 특성 아이디|PA_CROP_SPE_ID|10|필수|PA130201|주산지 작물별 특성 아이디(활용가이드 하단첨부 참고)|


### 출력결과(Response Element)

|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|---|---|---|---|---|---|
|결과코드|resultCode|2|필수|0|결과코드|
|결과메시지|resultMsg|50|필수|OK|결과메시지|
|한 페이지 결과 수|numOfRows|4|필수|10|한 페이지 결과 수|
|페이지 번호|pageNo|4|필수|1|페이지번호|
|전체 결과 수|totalCount|4|필수|3|전체 결과 수|
|데이터 타입|dataType|4|필수|XML|응답자료형식 (XML/JSON)|
|연원일|ymd|10|필수|20161201|일통계 날짜|
|지역 아이디|areaId|10|필수|4827000001|지역 아이디|
|지역 이름|areaName|50|필수|평택|지역 이름|
|작물 명|paCropName|20|필수|무|주산지 작물 이름|
|작물별 특성아이디|paCropSpeId|10|필수|PA020101|작물별 특성 아이디|
|작물별 특성 이름|paCropSpeName|20|필수|봄|주산지 작물별 특성 이름|
|일 평균기온|dayAvgTa|5|필수|24.5|일 평균기온|
|일 최고기온|dayMaxTa|5|필수|30|일 최고기온|
|일 최저기온|dayMinTa|5|필수|22|일 최저기온|
|일 평균상대습도|dayAvgRhm|5|필수|85|일 평균상대습도|
|일 최저상대습도|dayMinRhm|5|필수|60|일 최저상대습도|
|일 강수량|daySumRn|5|필수|25|일 강수량|
|일 평균풍속|dayAvgWs|5|필수|4|일 평균풍속|
|일 누적일조시간|daySumSs|5|필수|4|일 누적일조시간|
|특보 발효 여부|wmCount|4|필수|1|과거특보 발효여부(0:없음, 1:있음)|
|특보 코드|wmCd|8|필수|W2|특보 코드|


In [60]:
import time
from pprint import pprint

import numpy as np
import pandas as pd

# DC
import requests
from bs4 import BeautifulSoup

# about file format
import xmltodict

# 'auth' is included in gitignore.
import sys

sys.path.append("../Import")
import auth

authkey = auth.authkey["FmlandWthrInfo_dec"]

# 작물명을 넣으면 지역코드, 작물특성코드 반환하는 함수

- 작물명  →  (지역코드(AREA_ID), 작물특성코드(PA_CROP_SPE_ID))  ⇒  FmlandWthrInfo

## 문제(?)사항

1. 같은 작물에 복수의 작물_특성이 존재함.
    - 작물이 '호박'인 경우에 작물_특성으로 ['애호박', '쥬키니']가 있음.
2. 같은 작물 또는 같은 작물_특성에서 복수의 지역이 존재함.

- 위의 두 이유로 기상데이터를 어떻게 산출(계산?)하는 것이 예측에 좋을지 고민해보아야함. 

### 문제 해결 방안

- 문제1
    1. 모든 작물_특성을 사용.

- 문제2
    1. 모든 지역의 기상 데이터를 평균으로 계산
    2. 모든 지역 중 생산량 상위 n개 또는 n%를 평균 또는 가중치를 부여하여 계산


### 우리팀의 이번 프로젝트에서 선택한 방안과 그 이유

- 선택한 방안

- 선택한 이유

In [80]:
# 작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05

df = pd.read_excel("../data/작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05.xlsx")
df

,지역_아이디,지역_이름,주산지_작물_특성_아이디,주산지_작물_특성_이름,주산지_작물명
0,4211000000,춘천,PA360101,애호박,호박
1,4211000000,춘천,PA360201,쥬키니,호박
2,4272000000,홍천,PA360101,애호박,호박
3,4272000000,홍천,PA360201,쥬키니,호박
4,4278000000,철원,PA360201,쥬키니,호박
...,...,...,...,...,...
676,5011000000,제주,PA020301,가을,감자
677,5013000000,서귀포,PA020301,가을,감자
678,4689000000,완도,PA010101,-,감귤
679,5011000000,제주,PA010101,-,감귤


In [87]:
df[df["주산지_작물명"] == "양배추"]['지역_아이디'].nunique()

24

# 지역코드 → 농업기상정보

- ? → <u>**지역코드 → 농업기상정보**</u>


In [61]:
# api test
url = "http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics"
params = {
    "serviceKey": authkey,
    "pageNo": "1",
    "numOfRows": "10",
    "dataType": "XML",
    "ST_YMD": "20161201",
    "ED_YMD": "20161201",
    "AREA_ID": "4122000000",
    "PA_CROP_SPE_ID": "PA130201",
}

time.sleep(0.2)
response = requests.get(url, params=params)
if response.status_code == 200:
    soup = BeautifulSoup(response.text.encode("utf-8"), "xml")
    soup = soup.find_all("item")[0]

    # soup to dict
    parsed_dict = xmltodict.parse(str(soup))

    # dict to pd.df, check pd.df
    display(pd.DataFrame(parsed_dict).T.reset_index(drop=True))

    # check columns
    print(pd.DataFrame(parsed_dict).T.columns.to_list())

    columns = list(parsed_dict["item"].keys())
    print(columns)

    # set rename_columns, rename_dict
    rename_columns = [
        "Area_ID",
        "Area_Name",
        "DayAvg_RelativeHumidity",
        "DayAvg_Temperature",
        "dayAvg_WindSpeed",
        "DayMax_Temperature",
        'DayMin_RelativeHumidity',
        'DayMin_Temperature',
        'DaySum_Rainfall',
        'daySum_Sunshine',
        'Crop_Name',
        'CropSpecific_ID',
        'CropSpecific_Name',
        'Warning_Code',
        'Warning_Count',
        'YearMonthDay',
    ]
    rename_dict = {}
    for i, j in zip(columns, rename_columns):
        rename_dict[i] = j
    pprint(rename_dict)

,areaId,areaName,dayAvgRhm,dayAvgTa,dayAvgWs,dayMaxTa,dayMinRhm,dayMinTa,daySumRn,daySumSs,paCropName,paCropSpeId,paCropSpeName,wrnCd,wrnCount,ymd
0,4122000000,평택,59,5,3,9,19,0,0,0,무,PA130201,봄,None,0,2016-12-01 00:00:00


['areaId', 'areaName', 'dayAvgRhm', 'dayAvgTa', 'dayAvgWs', 'dayMaxTa', 'dayMinRhm', 'dayMinTa', 'daySumRn', 'daySumSs', 'paCropName', 'paCropSpeId', 'paCropSpeName', 'wrnCd', 'wrnCount', 'ymd']
['areaId', 'areaName', 'dayAvgRhm', 'dayAvgTa', 'dayAvgWs', 'dayMaxTa', 'dayMinRhm', 'dayMinTa', 'daySumRn', 'daySumSs', 'paCropName', 'paCropSpeId', 'paCropSpeName', 'wrnCd', 'wrnCount', 'ymd']
{'areaId': 'Area_ID',
 'areaName': 'Area_Name',
 'dayAvgRhm': 'DayAvg_RelativeHumidity',
 'dayAvgTa': 'DayAvg_Temperature',
 'dayAvgWs': 'dayAvg_WindSpeed',
 'dayMaxTa': 'DayMax_Temperature',
 'dayMinRhm': 'DayMin_RelativeHumidity',
 'dayMinTa': 'DayMin_Temperature',
 'daySumRn': 'DaySum_Rainfall',
 'daySumSs': 'daySum_Sunshine',
 'paCropName': 'paCrop_Name',
 'paCropSpeId': 'paCropSpe_ID',
 'paCropSpeName': 'paCropSpe_Name',
 'wrnCd': 'Warning_Code',
 'wrnCount': 'Warning_Count',
 'ymd': 'YearMonthDay'}


In [67]:
def Get_df_AgriWeather(
    authkey: str,
    pageNo: int | str,
    search_Year_Period: tuple,
    AREA_ID: str,
    PA_CROP_SPE_ID: str,
) -> pd.DataFrame | int:

    """
    API에서 농업기상정보를 가져오는 함수
    단순하게 API에서 요구하는 'Page_Size'를 제외한 요청변수를 인자로 받음.
    """

    url = "http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics"
    params = {
        "serviceKey": authkey,
        "pageNo": pageNo,
        "numOfRows": 100,
        "dataType": "XML",
        "ST_YMD": f"{search_Year_Period[0]}0101",
        "ED_YMD": f"{search_Year_Period[1]}1231",
        "AREA_ID": AREA_ID,
        "PA_CROP_SPE_ID": PA_CROP_SPE_ID,
    }

    # API
    time.sleep(0.2)
    response = requests.get(url, params)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "xml")
        soup = soup.find_all("item")
        s = soup[-1]

        # col
        col = xmltodict.parse(str(s))
        col = list(col["item"].keys())

        # return
        df = pd.DataFrame()

        # # no
        # no = int(s.find("no").get_text())
        # if no >= 301:
        #     roof_size = no % 100
        # else:
        #     roof_size = 100

        for i in range(0, 100):
            try:
                s = soup[i]
                temp_dict = {"temp_idx": {}}
            except:
                continue

            for j in col:
                try:
                    v = s.find(j).get_text()
                    temp_dict["temp_idx"][j] = v
                except:
                    pass

            df_new = pd.DataFrame(temp_dict).T
            df = pd.concat([df, df_new]).reset_index(drop=True)

        return df[col]

    else:
        print(response.status_code)
        return response.status_code


def Get_df_PeriodAgriWeather(
    authkey: str,
    search_Year_Period: tuple,
    AREA_ID: str,  
    PA_CROP_SPE_ID: str,
    rename_col=True,
) -> pd.DataFrame:

    """
    'Get_df_AgriWeather' 함수를 활용하여, 더 편하게(?) 농업기상정보를 가져오는 함수
    """

    df = pd.DataFrame()
    for y in range(search_Year_Period[0], search_Year_Period[1] + 1):
        for p in range(1, 5):
            df_new = Get_df_AgriWeather(
                authkey=authkey, pageNo=p, search_Year_Period=search_Year_Period, AREA_ID=AREA_ID, PA_CROP_SPE_ID=PA_CROP_SPE_ID
            )
            df = pd.concat([df, df_new])

    # 대략적인 전처리 - 시간 지우기
    df["ymd"] = df["ymd"].str.replace(" 00:00:00","")

    # rename_col
    if rename_col == True:
        rename_dict = {
            "ymd": "YearMonthDay",
            "areaId": "Area_ID",
            "areaName": "Area_Name",
            "paCropName": "Crop_Name",
            "paCropSpeId": "CropSpecific_ID",
            "paCropSpeName": "CropSpecific_Name",
            "dayAvgTa": "DayAvg_Temperature",
            "dayMaxTa": "DayMax_Temperature",
            "dayMinTa": "DayMin_Temperature",
            "dayAvgRhm": "DayAvg_RelativeHumidity",
            "dayMinRhm": "DayMin_RelativeHumidity",
            "daySumRn": "DaySum_Rainfall",
            "dayAvgWs": "dayAvg_WindSpeed",
            "daySumSs": "daySum_Sunshine",
            "wrnCount": "Warning_Count",
            "wrnCd": "Warning_Code",
        }
        df = df[list(rename_dict.keys())].rename(columns=rename_dict)

    return df


display(
    Get_df_PeriodAgriWeather(
        authkey=authkey,
        search_Year_Period=(2018, 2019),
        AREA_ID="4122000000",
        PA_CROP_SPE_ID="PA130201"
    )
)

,YearMonthDay,Area_ID,Area_Name,paCrop_Name,paCropSpe_ID,paCropSpe_Name,DayAvg_Temperature,DayMax_Temperature,DayMin_Temperature,DayAvg_RelativeHumidity,DayMin_RelativeHumidity,DaySum_Rainfall,dayAvg_WindSpeed,daySum_Sunshine,Warning_Count,Warning_Code
0,2018-01-01,4122000000,평택,무,PA130201,봄,-1,3,-7,62,31,0,1,0,0,
1,2018-01-02,4122000000,평택,무,PA130201,봄,-1,2,-6,59,29,0,1,0,0,
2,2018-01-03,4122000000,평택,무,PA130201,봄,-3,0,-5,47,32,0,2,0,0,
3,2018-01-04,4122000000,평택,무,PA130201,봄,-3,0,-7,51,33,0,1,0,0,
4,2018-01-05,4122000000,평택,무,PA130201,봄,-2,1,-6,70,47,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-01-07,4122000000,평택,무,PA130201,봄,-2,4,-9,68,43,0,1,0,1,D2
96,2019-01-08,4122000000,평택,무,PA130201,봄,-3,1,-7,53,24,0,2,0,1,D2
97,2019-01-09,4122000000,평택,무,PA130201,봄,-4,1,-10,42,17,0,1,0,1,D2
98,2019-01-10,4122000000,평택,무,PA130201,봄,-1,1,-4,69,38,0,0,0,1,D2
